In [4]:
!pip install opencv-python

     ---------------------------------------- 38.6/38.6 MB 3.0 MB/s eta 0:00:00


In [1]:
import cv2

In [2]:
import os
import json
import pandas as pd

In [5]:
with open('s1.json','r') as json_file:
    data=json.load(json_file)
    annotations=data.get('shapes',[])

In [9]:
labels = []
points = []
for annotation in annotations:
    label = annotation.get('label', '')
    points_data = annotation.get('points', [])
    
    labels.append(label)
    points.append(points_data)

In [11]:
df = pd.DataFrame({'Label': labels, 'Points': points})


In [17]:
df.iloc[0]['Points']

[[274.99367088607596, 278.4810126582278],
 [678.7911392405063, 754.4303797468355]]

In [27]:
df['Points'].apply(coordinates)

0     (274.99367088607596, 678.7911392405063, 278.48...
1     (697.7784810126582, 791.4493670886077, 615.189...
2     (572.4620253164558, 650.9430379746834, 796.202...
3     (342.0822784810126, 542.0822784810127, 889.873...
4     (433.2215189873418, 771.1962025316457, 1077.21...
5     (791.4493670886077, 915.5, 1077.2151898734178,...
6     (15.5, 173.72784810126586, 812.6582278481012, ...
7     (25.62658227848101, 134.48734177215192, 670.88...
8     (6.639240506329088, 159.80379746835445, 489.87...
9     (71.19620253164555, 242.08227848101262, 79.746...
10    (247.14556962025313, 359.80379746835445, 26.58...
11    (390.1835443037975, 506.6392405063291, 22.7848...
12    (621.8291139240507, 702.8417721518988, 13.9240...
13    (771.1962025316457, 919.2974683544303, 122.784...
14    (838.2848101265822, 906.6392405063291, 311.392...
15    (797.7784810126582, 923.0949367088608, 405.063...
Name: Points, dtype: object

In [22]:
def coordinates(s):
    x1=s[0][0]
    x2=s[0][1]
    y1=s[1][0]
    y2=s[1][1]
    
    return x1,y1,x2,y2

In [28]:
def extract_values(row):
    return pd.Series([value for sublist in row for value in sublist])

# Apply the function to create new columns
df_new = df['Points'].apply(extract_values)

In [33]:
data=pd.concat([df_new,df],axis=1)

In [36]:
data.drop(columns=['Points'],inplace=True)

In [37]:
data

,0,1,2,3,Label
0,274.993671,278.481013,678.791139,754.430380,mythological creature
1,697.778481,615.189873,791.449367,1045.569620,mythological creature
2,572.462025,796.202532,650.943038,1032.911392,mythological creature
3,342.082278,889.873418,542.082278,1056.962025,animal
4,433.221519,1077.215190,771.196203,1192.405063,animal
5,791.449367,1077.215190,915.500000,1182.278481,animal
6,15.500000,812.658228,173.727848,1192.405063,mythological creature
7,25.626582,670.886076,134.487342,763.291139,animal
8,6.639241,489.873418,159.803797,660.759494,mythological creature
9,71.196203,79.746835,242.082278,282.278481,mythological creature


In [43]:
def extract_roi(image, bounding_box):
    x1, y1 = map(int, bounding_box[0])
    x2, y2 = map(int, bounding_box[1])
    roi = image[y1:y2, x1:x2]
    return roi

def extract_rgb_values(roi):
    rgb_values = roi.reshape(-1, 3) 
    return rgb_values.flatten()

features = [] 
labels1 = [] 

In [44]:
image=cv2.imread('s1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [45]:
for annotation in annotations:
    label = annotation['label']
    bounding_box = annotation['points']

    roi = extract_roi(image, bounding_box)
    rgb_values = extract_rgb_values(roi)

    features.append(rgb_values)
    labels1.append(label)

In [56]:
features
#normalize these values

[array([211, 189,  88, ..., 191, 214, 162], dtype=uint8),
 array([198, 219, 178, ...,  61,  71,  63], dtype=uint8),
 array([218, 230, 190, ..., 191, 210, 155], dtype=uint8),
 array([61, 47, 62, ..., 99, 59, 67], dtype=uint8),
 array([205, 217, 177, ...,  58,  56,  70], dtype=uint8),
 array([220, 230, 206, ..., 211, 222, 190], dtype=uint8),
 array([178,  88,  80, ..., 193, 211, 161], dtype=uint8),
 array([233, 228, 186, ..., 208, 146, 125], dtype=uint8),
 array([103,  95,  76, ..., 223, 213, 102], dtype=uint8),
 array([225, 216, 185, ..., 250, 235, 178], dtype=uint8),
 array([216, 211, 155, ..., 221, 216, 160], dtype=uint8),
 array([220, 213, 141, ..., 116,  64,  66], dtype=uint8),
 array([227, 225, 176, ...,  50,  48,  51], dtype=uint8),
 array([217, 207, 145, ..., 228, 224, 187], dtype=uint8),
 array([ 41,  42,  46, ..., 228, 224, 177], dtype=uint8),
 array([119, 165, 242, ...,  58,  58,  66], dtype=uint8)]

In [48]:
data

,0,1,2,3,Label
0,274.993671,278.481013,678.791139,754.430380,mythological creature
1,697.778481,615.189873,791.449367,1045.569620,mythological creature
2,572.462025,796.202532,650.943038,1032.911392,mythological creature
3,342.082278,889.873418,542.082278,1056.962025,animal
4,433.221519,1077.215190,771.196203,1192.405063,animal
5,791.449367,1077.215190,915.500000,1182.278481,animal
6,15.500000,812.658228,173.727848,1192.405063,mythological creature
7,25.626582,670.886076,134.487342,763.291139,animal
8,6.639241,489.873418,159.803797,660.759494,mythological creature
9,71.196203,79.746835,242.082278,282.278481,mythological creature


In [55]:
for arr in features:
    print(len(arr))

576912
121260
55224
100200
116610
39060
180120
30411
78489
104139
52080
69600
52002
77700
17748
88830


In [1]:
#2d array into 1d array by flatten. size depends on the no. of pixels in each image.

In [12]:
def extract_roi(image, bounding_box):
    x1, y1 = map(int, bounding_box[0])
    x2, y2 = map(int, bounding_box[1])
    roi = image[y1:y2, x1:x2]
    return roi

def extract_rgb_values(roi):
    rgb_values = roi.reshape(-1, 3) 
    return rgb_values.flatten()

features = [] 
labels1 = [] 

In [14]:
import os
import json

# Specify the root directory of your folder structure
root_directory = r'C:\Users\acer\OneDrive\Desktop\folk_art_image'

# Loop through each subfolder (s1 to s13)
for subfolder in range(1, 14):
    subfolder_name = f's{subfolder}'

    # Path to the current subfolder
    subfolder_path = os.path.join(root_directory, subfolder_name)

    # Loop through each file in the subfolder
    for filename in os.listdir(subfolder_path):
        # Check if the file is a JSON file
        if filename.endswith('.json'):
            json_filepath = os.path.join(subfolder_path, filename)

            # Read the JSON file
            with open(json_filepath, 'r') as json_file:
                json_data = json.load(json_file)
                annotations = json_data.get('shapes', [])

            # Load and process the image
            image = cv2.imread(f'{subfolder_name}.img')
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Process each annotation
            for annotation in annotations:
                label = annotation['label']
                bounding_box = annotation['points']

                roi = extract_roi(image, bounding_box)
                rgb_values = extract_rgb_values(roi)

                features.append(rgb_values)
                labels1.append(label)



TypeError: 'NoneType' object is not subscriptable

In [9]:
for subfolder in range(1, 14):
    subfolder_name = f's{subfolder}'
    print(f'{subfolder_name}.img')

s1.img
s2.img
s3.img
s4.img
s5.img
s6.img
s7.img
s8.img
s9.img
s10.img
s11.img
s12.img
s13.img


In [19]:
import os
import cv2
import json

# Assuming you have the extract_roi and extract_rgb_values functions defined
def extract_roi(image, bounding_box):
    x1, y1 = map(int, bounding_box[0])
    x2, y2 = map(int, bounding_box[1])
    roi = image[y1:y2, x1:x2]
    return roi

def extract_rgb_values(roi):
    rgb_values = roi.reshape(-1, 3) 
    return rgb_values.flatten()

# Assuming you have the features and labels1 variables declared
features = []
labels1 = []

# Specify the root directory of your folder structure
root_directory = r'C:\Users\acer\OneDrive\Desktop\folk_art_image'
folder_name = 's1'

# Construct the folder path
folder_path = os.path.join(root_directory, folder_name)

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JPG file
    if filename.endswith('.jpg'):
        # Construct the file paths for the JPG and JSON files
        jpg_filepath = os.path.join(folder_path, filename)
        json_filepath = os.path.join(folder_path, f"{filename.split('.')[0]}.json")

        # Check if the corresponding JSON file exists
        if os.path.exists(json_filepath):
            # Read the JSON file
            with open(json_filepath, 'r') as json_file:
                data = json.load(json_file)
                annotations = data.get('shapes', [])

            # Load and process the image
            image = cv2.imread(jpg_filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Process each annotation
            for annotation in annotations:
                label = annotation['label']
                bounding_box = annotation['points']

                roi = extract_roi(image, bounding_box)
                rgb_values = extract_rgb_values(roi)

                features.append(rgb_values)
                labels1.append(label)



In [ ]:
df = pd.DataFrame({
    'labels': labels1,
    'features': features
})
df

In [33]:
df

,labels,features
0,mythological creature,"[211, 189, 88, 222, 201, 96, 214, 193, 88, 214..."
1,mythological creature,"[198, 219, 178, 190, 212, 165, 188, 211, 157, ..."
2,mythological creature,"[218, 230, 190, 189, 202, 158, 204, 220, 173, ..."
3,animal,"[61, 47, 62, 65, 56, 51, 119, 81, 78, 58, 50, ..."
4,animal,"[205, 217, 177, 207, 219, 179, 208, 220, 180, ..."
5,animal,"[220, 230, 206, 218, 227, 206, 222, 232, 208, ..."
6,mythological creature,"[178, 88, 80, 233, 213, 188, 255, 241, 210, 17..."
7,animal,"[233, 228, 186, 234, 229, 187, 235, 230, 188, ..."
8,mythological creature,"[103, 95, 76, 230, 221, 154, 234, 230, 159, 23..."
9,mythological creature,"[225, 216, 185, 224, 217, 163, 223, 216, 162, ..."


In [34]:
import os
import cv2
import json

# Assuming you have the extract_roi and extract_rgb_values functions defined
def extract_roi(image, bounding_box):
    x1, y1 = map(int, bounding_box[0])
    x2, y2 = map(int, bounding_box[1])
    roi = image[y1:y2, x1:x2]
    return roi

def extract_rgb_values(roi):
    rgb_values = roi.reshape(-1, 3) 
    return rgb_values.flatten()

# Assuming you have the features and labels1 variables declared
features = []
labels1 = []

# Specify the root directory of your folder structure
root_directory = r'C:\Users\acer\OneDrive\Desktop\folk_art_image'
folder_name = 's2'

# Construct the folder path
folder_path = os.path.join(root_directory, folder_name)

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JPG file
    if filename.endswith('.jpg'):
        # Construct the file paths for the JPG and JSON files
        jpg_filepath = os.path.join(folder_path, filename)
        json_filepath = os.path.join(folder_path, f"{filename.split('.')[0]}.json")

        # Check if the corresponding JSON file exists
        if os.path.exists(json_filepath):
            # Read the JSON file
            with open(json_filepath, 'r') as json_file:
                data = json.load(json_file)
                annotations = data.get('shapes', [])

            # Load and process the image
            image = cv2.imread(jpg_filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Process each annotation
            for annotation in annotations:
                label = annotation['label']
                bounding_box = annotation['points']

                roi = extract_roi(image, bounding_box)
                rgb_values = extract_rgb_values(roi)

                features.append(rgb_values)
                labels1.append(label)



In [1]:
import os
import cv2
import json
import pandas as pd
import numpy as np

# Assuming you have the extract_roi and extract_rgb_values functions defined
def extract_roi(image, bounding_box):
    x1, y1 = map(int, bounding_box[0])
    x2, y2 = map(int, bounding_box[1])
    roi = image[y1:y2, x1:x2]
    return roi

def extract_rgb_values(roi):
    rgb_values = roi.reshape(-1, 3) 
    return rgb_values.flatten()

# Assuming you have the features and labels1 variables declared
all_features = []
all_labels = []

# Specify the root directory of your folder structure
root_directory = r'C:\Users\acer\OneDrive\Desktop\folk_art_image'

# Loop through each folder (s1 to s13)
for subfolder in range(1, 14):
    folder_name = f's{subfolder}'
    folder_path = os.path.join(root_directory, folder_name)

    # Assuming you have the features and labels1 variables declared within the loop
    features = []
    labels1 = []

    # Loop through each file in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is a JPG file
        if filename.endswith('.jpg'):
            # Construct the file paths for the JPG and JSON files
            jpg_filepath = os.path.join(folder_path, filename)
            json_filepath = os.path.join(folder_path, f"{filename.split('.')[0]}.json")

            # Check if the corresponding JSON file exists
            if os.path.exists(json_filepath):
                # Read the JSON file
                with open(json_filepath, 'r') as json_file:
                    data = json.load(json_file)
                    annotations = data.get('shapes', [])

                # Load and process the image
                image = cv2.imread(jpg_filepath)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Process each annotation
                for annotation in annotations:
                    label = annotation['label']
                    bounding_box = annotation['points']

                    roi = extract_roi(image, bounding_box)
                    rgb_values = extract_rgb_values(roi)

                    features.append(rgb_values)
                    labels1.append(label)

    # Append the features and labels for the current folder to the overall lists
    all_features.extend(features)
    all_labels.extend(labels1)

# Create a DataFrame for all folders
df = pd.DataFrame({
    'labels': all_labels,
    'features': all_features
})


In [40]:
animal_df=df[df['labels']=='animal']

In [41]:
animal_df

,labels,features
3,animal,"[61, 47, 62, 65, 56, 51, 119, 81, 78, 58, 50, ..."
4,animal,"[205, 217, 177, 207, 219, 179, 208, 220, 180, ..."
5,animal,"[220, 230, 206, 218, 227, 206, 222, 232, 208, ..."
7,animal,"[233, 228, 186, 234, 229, 187, 235, 230, 188, ..."
15,animal,"[119, 165, 242, 119, 166, 236, 124, 172, 238, ..."
...,...,...
166,animal,"[7, 9, 21, 10, 8, 19, 6, 2, 16, 4, 0, 12, 6, 0..."
178,animal,"[242, 209, 42, 236, 203, 38, 232, 199, 36, 231..."
206,animal,"[0, 0, 7, 1, 2, 6, 0, 0, 2, 3, 4, 6, 0, 3, 9, ..."
211,animal,"[135, 187, 235, 134, 186, 234, 135, 187, 235, ..."


In [50]:
animal_df=animal_df.reset_index(drop=True)

In [52]:

max_length = animal_df['features'].apply(len).max()

# Create a new DataFrame with columns for each value in the lists
new_columns = [f'feature_{i+1}' for i in range(max_length)]
new_df = pd.DataFrame(animal_df['features'].to_list(), columns=new_columns)

# Concatenate the new DataFrame with the 'label' column from the original DataFrame
result_df = pd.concat([animal_df['labels'], new_df], axis=1)

# Fill NaN values with 0
result_df = result_df.fillna(0)


In [53]:
result_df

,labels,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1092987,feature_1092988,feature_1092989,feature_1092990,feature_1092991,feature_1092992,feature_1092993,feature_1092994,feature_1092995,feature_1092996
0,animal,61,47,62,65,56,51,119,81,78,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,animal,205,217,177,207,219,179,208,220,180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,animal,220,230,206,218,227,206,222,232,208,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,animal,233,228,186,234,229,187,235,230,188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,animal,119,165,242,119,166,236,124,172,238,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,animal,7,9,21,10,8,19,6,2,16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63,animal,242,209,42,236,203,38,232,199,36,...,38.0,21.0,31.0,41.0,16.0,12.0,13.0,46.0,32.0,31.0
64,animal,0,0,7,1,2,6,0,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,animal,135,187,235,134,186,234,135,187,235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
result_df.to_csv('animals.csv', index=False)

In [2]:
df

,labels,features
0,mythological creature,"[211, 189, 88, 222, 201, 96, 214, 193, 88, 214..."
1,mythological creature,"[198, 219, 178, 190, 212, 165, 188, 211, 157, ..."
2,mythological creature,"[218, 230, 190, 189, 202, 158, 204, 220, 173, ..."
3,animal,"[61, 47, 62, 65, 56, 51, 119, 81, 78, 58, 50, ..."
4,animal,"[205, 217, 177, 207, 219, 179, 208, 220, 180, ..."
...,...,...
208,mythological creature,"[27, 10, 16, 61, 20, 34, 125, 57, 78, 189, 95,..."
209,tree,"[1, 0, 6, 1, 0, 6, 1, 0, 5, 1, 0, 5, 1, 0, 5, ..."
210,mythological creature,"[137, 177, 228, 140, 182, 232, 143, 187, 236, ..."
211,animal,"[135, 187, 235, 134, 186, 234, 135, 187, 235, ..."


In [4]:
for array in df['features']:
    print(len(array))

576912
121260
55224
100200
116610
39060
180120
30411
78489
104139
52080
69600
52002
77700
17748
88830
32391
92826
19224
249804
26892
27666
63813
28482
72450
302331
100548
472230
197760
198720
90870
82713
87822
96099
1110375
28737
68970
339660
1515360
165000
307980
320850
184464
35145
47724
61359
228528
66780
172872
487200
192444
755811
210018
68076
64449
51150
226353
122139
85275
10080
43824
481572
144480
207018
127224
643260
211422
86328
98766
114036
93912
21456
26910
24444
50778
33300
44352
47058
54531
289476
164160
172590
363636
266760
176280
393360
216216
228177
180075
381036
342258
383274
333504
68310
44100
373527
459264
353925
293562
663390
271170
512082
546264
473040
619740
644952
90288
402336
721230
472032
623040
602538
183918
312000
10431
14271
352800
99696
71136
297672
250920
144837
974073
337110
40368
201432
267900
57084
223260
153270
473373
480816
92547
232650
59904
192324
282183
113022
181662
225720
223668
64512
252495
336690
212868
93495
93771
88389
167049
126555
251544
2

In [5]:
df

,labels,features
0,mythological creature,"[211, 189, 88, 222, 201, 96, 214, 193, 88, 214..."
1,mythological creature,"[198, 219, 178, 190, 212, 165, 188, 211, 157, ..."
2,mythological creature,"[218, 230, 190, 189, 202, 158, 204, 220, 173, ..."
3,animal,"[61, 47, 62, 65, 56, 51, 119, 81, 78, 58, 50, ..."
4,animal,"[205, 217, 177, 207, 219, 179, 208, 220, 180, ..."
...,...,...
208,mythological creature,"[27, 10, 16, 61, 20, 34, 125, 57, 78, 189, 95,..."
209,tree,"[1, 0, 6, 1, 0, 6, 1, 0, 5, 1, 0, 5, 1, 0, 5, ..."
210,mythological creature,"[137, 177, 228, 140, 182, 232, 143, 187, 236, ..."
211,animal,"[135, 187, 235, 134, 186, 234, 135, 187, 235, ..."
